# Tree to png

In [2]:
from treelib import Tree, Node
import pydot as pd

In [2]:
t = Tree()
t.create_node('root','root')
t.create_node('ch1','ch1',parent='root')
t.create_node('ch2','ch2',parent='root')

Node(tag=ch2, identifier=ch2, data=None)

In [3]:
t.to_graphviz('test.gv')
dot = pd.graph_from_dot_file('test.gv')[0]

In [4]:
dot.write_png('test.png')

# Optimizing score and heuristic

starting with evaluating the performance of regex

In [4]:
import re
from time import perf_counter_ns
from model.grid import Grid

In [4]:
reg = re.compile("(1(?!1{3,})|0(?!0{3,})){4,}")

avg = 0
max = 0
for i in range(4000000):
    start = perf_counter_ns()
    j = 0
    matches = reg.finditer("00001201010")
    for match in matches:
        j += 1
    d = (perf_counter_ns() - start) / 1e6
    avg = (avg +  d)/ 2
    if max < d: max = d
print("Average =",avg,"ms")
print("Max =",max,"ms")

Average = 0.005135909966863656 ms
Max = 35.9908 ms


The performance of regex is in the range of microseconds. It is not the problem, these unrealistic numbers are coming from somewhere else in the score and heuristic functions

## using regex for scores

In [16]:
g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_score()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [1,2,3,4,5,6,7,8,9,10,11,12,13]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

avg = 0.23693513147081263 ms.
max = 3.5371 ms


Minor optimization: which representation of the elements is more efficient at conversion to strings.

In [18]:
avgchr = 0
avgint = 0
for i in range(10000000):
    t = perf_counter_ns()
    chr(65)
    avgchr = ((avgchr + (perf_counter_ns() - t) / 1e6)) / 2
    t = perf_counter_ns()
    str(1)
    avgint = (avgint + ((perf_counter_ns() - t) / 1e6)) /2
print(avgchr)
print(avgint)

0.00016249998956445748
0.0003000015735742899


using bytes is better than ints

In [1]:
from model.grid import Grid
from time import perf_counter_ns

g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_score()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [i+1 for i in range(13)]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

avg = 0.20091451739831348 ms.
max = 1.103 ms


## replacing regex with manual string search

In [1]:
from model.grid import Grid
from time import perf_counter_ns

g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_score()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [i+1 for i in range(13)]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

avg = 0.14025233685007482 ms.
max = 2.1745 ms


## Heuristic as is

In [1]:
from model.grid import Grid
from time import perf_counter_ns

g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_heuristic_value()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [i+1 for i in range(13)]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

avg = 0.2602384100792934 ms.
max = 3.6305 ms
1.822 seconds.
12.752 seconds.
89.262 seconds.
624.832 seconds.
4373.827 seconds.
30616.789 seconds.
214317.521 seconds.
1500222.647 seconds.
10501558.527 seconds.
73510909.687 seconds.
514576367.806 seconds.
3602034574.639 seconds.
25214242022.474 seconds.


change heuristic to manual string search

In [2]:
from model.grid import Grid
from time import perf_counter_ns

g = Grid()
avg = 0
max = 0
for i in range(1000):
    start = perf_counter_ns()
    score = g.get_heuristic_value()
    duration = (perf_counter_ns() - start) / 1e6
    avg  = (avg + duration) / 2
    if duration > max: max = duration
print("avg =",avg,"ms.")
print("max =",max,"ms")
# how much it would take to evaluate for each of these depth roughly
time = avg
k = [i+1 for i in range(13)]
for i in k : print(round(time * 7 ** i,3) , "seconds.")

avg = 1.9824533947657281 ms.
max = 9.1057 ms
13.877 seconds.
97.14 seconds.
679.982 seconds.
4759.871 seconds.
33319.094 seconds.
233233.659 seconds.
1632635.616 seconds.
11428449.313 seconds.
79999145.188 seconds.
559994016.317 seconds.
3919958114.221 seconds.
27439706799.55 seconds.
192077947596.849 seconds.


## Changes

- Heuristic will remain with regex
- Score will use manual string search
- Grid will be represented as bytes